In [1]:
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k', binary=True, pos_ne_divider=0)

load_builtinTrue0
load_from_fileTrue0


In [3]:
# trainset, testset = train_test_split(data, test_size=.25, binary=True, pos_ne_divider=2.5, random_state=100)
trainset, testset = train_test_split(data, test_size=.25, random_state=100)

construct_trainsetTrue0
construct_testsetTrue0


In [4]:
trainset.global_mean

1.0

In [5]:
algo = SVDpp(random_state=100)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE


KeyboardInterrupt: 

In [7]:
accuracy.rmse(predictions)
accuracy.mae(predictions)
accuracy.fcp(predictions)

RMSE: 0.4182
MAE:  0.1793
FCP:  0.1216


0.12164199067103677

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=1):
    '''Return precision and recall at k metrics for each user.'''

#     # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, _, est, _, true_r in predictions:
        user_est_true[uid].append((est, true_r))
#     for index, row in predictions.iterrows():
#         user_est_true[row['user_id']].append((row['estimated_Ratings'], row['true_r']))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [ ]:
from collections import defaultdict
precisions, recalls = precision_recall_at_k(df, k=5, threshold=0.05)
# precisions
# recalls
rc = [r for _,r in recalls.items()]
rc_mean = sum(rc)/len(rc)
pr = [r for _,r in precisions.items()]
pr_mean = sum(pr)/len(pr)